### **Instalando e importando as bibliotecas necessárias**

In [277]:
%pip install PyOpenGL
%pip install glfw
%pip install pyglm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\kasat\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\kasat\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: pyglm in c:\users\kasat\appdata\local\packages\pythonsoftwarefoundation.python.3.11_qbz5n2kfra8p0\localcache\local-packages\python311\site-packages (2.8.0)




[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\kasat\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [278]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
import glm

### **Inicializando a janela**

In [279]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
window = glfw.create_window(720, 600, "Avião em voo", None, None)

if (window == None):
    print("Failed to create GLFW window")
    glfwTerminate()
    
glfw.make_context_current(window)

### **Shaders**
Defininindo os programas Vertex e Fragment Shaders e requisitando slot para a GPU para eles. Após isso, associamos nosso código-fonte aos slots associados.

In [280]:
vertex_code = """
    #version 330 core
    layout (location = 0) in vec3 position;
    uniform mat4 mat_transformation;
    
    void main() {
        gl_Position = mat_transformation * vec4(position, 1.0);
    }
"""

fragment_code = """
    #version 330 core
    uniform vec4 color;
    out vec4 FragColor;
    
    void main() {
        FragColor = color;
    }
"""

program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

Compilando nossos programas e associando ao programa principal

In [281]:
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")

glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

glAttachShader(program, vertex)
glAttachShader(program, fragment)

### Linkagem e Preparação de Dados para a GPU

Build do programa tornando ele default

In [282]:
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
glUseProgram(program)

Modelagem dos objetos em cena

In [283]:
# Para gerar uma novem uma ideia foi gerar alguns polígonos irregulares e junta-los
def geraPoligono3D(n_lados, tamanho_lados, espessura, deslocamento_z, rotacao):
    pontos = []
    offset_angulo = rotacao

    # definindo pontos do polígono
    for i in range(n_lados):
        x = tamanho_lados[i] * np.cos(offset_angulo)
        y = tamanho_lados[i] * np.sin(offset_angulo)
        pontos.append([x, y, deslocamento_z])
        offset_angulo += 2 * np.pi / n_lados

    # como é um objeto 3D, replicamos os pontos para gerar uma espessura, com um diferente valor de z
    copia_pontos = [[x, y, espessura + deslocamento_z] for x, y, _ in pontos]

    # fazendo o mesmo prpcesso para os vértices agora
    vertices = np.array(pontos + copia_pontos, dtype=np.float32)

    indices = []
    for i in range(n_lados):
        prox = (i + 1) % n_lados
        indices.extend([i, prox, n_lados + prox, n_lados + i])

    indices = np.array(indices, dtype=np.uint32)

    return vertices, indices

NUM_POLIGONOS = 5 # quantidade de polígonos que formarão a nuvem
poligonos_vertices = []
poligonos_indices = []
deslocamento_z = 0.0
indice_offset = 0

for _ in range(NUM_POLIGONOS):
    n_lados = np.random.randint(12,20) # os polígonos podem variar entre 5 e 10 faces
    tamanhos = np.random.uniform(1.5, 2.5, n_lados)
    rotacao = np.random.uniform(0, 2 * np.pi)
    vertices, indices = geraPoligono3D(n_lados, tamanhos, 0.5, deslocamento_z, rotacao)

    indices = indices + indice_offset  # Ajusta os índices para a nova posição no array
    poligonos_vertices.append(vertices)
    poligonos_indices.append(indices)

    indice_offset += len(vertices)  # Atualiza o deslocamento dos índices
    deslocamento_z -= 0.3  # O próximo polígono ficará atrás do anterior

# Converte para NumPy corretamente
poligonos_vertices = np.vstack(poligonos_vertices)
poligonos_indices = np.hstack(poligonos_indices)


Configuração VAO/VBO/EBO

In [284]:
VAO = glGenVertexArrays(1)
glBindVertexArray(VAO)

VBO = glGenBuffers(1)
glBindBuffer(GL_ARRAY_BUFFER, VBO)
glBufferData(GL_ARRAY_BUFFER, poligonos_vertices.nbytes, poligonos_vertices, GL_STATIC_DRAW)

EBO = glGenBuffers(1)
glBindBuffer(GL_ELEMENT_ARRAY_BUFFER, EBO)
glBufferData(GL_ELEMENT_ARRAY_BUFFER, poligonos_indices.nbytes, poligonos_indices, GL_STATIC_DRAW)

Associando variáveis do programa GLSL (Vertex Shader) com nossos dados

In [285]:
glVertexAttribPointer(0, 3, GL_FLOAT, False, 12, ctypes.c_void_p(0))
glEnableVertexAttribArray(0)

Definindo cores do projeto (mudar pra cá dps)

### **Capturando eventos de teclado e modificando variáveis para a matriz de transformação**

In [286]:
fator_escala = 0.5

def key_event(window,key,scancode,action,mods):
    global t_x, t_y, angulo, escala
    
    if key == 265: t_y += 0.01 #cima
    if key == 264: t_y -= 0.01 #baixo
    if key == 263: t_x -= 0.01 #esquerda
    if key == 262: t_x += 0.01 #direita

    if key == 65:  angulo += 0.1 # gira para a direita
    if key == 83:  angulo -= 0.1 # gira para a esquerda

    if key == 90: escala += 0.1 * fator_escala # aumenta escala 
    if key == 88: escala -= 0.1 * fator_escala # diminui escala

    if key == 82: 
        t_x = 0
        t_y = 0
        angulo = 0
        escala = 1


    
glfw.set_key_callback(window,key_event)

### **Funções Principais da Janela**

In [287]:
glfw.show_window(window)

In [288]:
import math

angulo = 0
t_x = 0
t_y = 0
escala = 1


def multiplica_matriz(a,b):
    m_a = a.reshape(4,4)
    m_b = b.reshape(4,4)
    m_c = np.dot(m_a,m_b)
    c = m_c.reshape(1,16)
    return c

# Matriz de projeção ortográfica (mapeia coordenadas 3D para a tela)
glEnable(GL_BLEND)
glBlendFunc(GL_SRC_ALPHA, GL_ONE_MINUS_SRC_ALPHA)
glEnable(GL_DEPTH_TEST)
glClearColor(0.53, 0.81, 0.92, 1.0)  # Cor de céu

# Matriz de projeção perspectiva
width, height = glfw.get_window_size(window)
proj = glm.perspective(glm.radians(60.0), width/height, 0.1, 100.0)

# Posição da câmera
view = glm.lookAt(
    glm.vec3(2, 2, 2),  # Posição da câmera
    glm.vec3(0, 0, 0),  # Ponto para onde olha
    glm.vec3(0, 1, 0)   # Vetor "para cima"
)

# Enviamos para o shader
loc_transform = glGetUniformLocation(program, "mat_transformation")
loc_color = glGetUniformLocation(program, "color")


# Loop principal
while not glfw.window_should_close(window):
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)

    # Animação de rotação
    angulo += 0.0001
    model = glm.mat4(1.0)
    model = glm.rotate(model, angulo, glm.vec3(0, 1, 0))  # Rotaciona em Y
    
    # Configura transformações
    mat_model = glm.mat4(1.0)
    mat_model = glm.translate(mat_model, glm.vec3(t_x, t_y, 0.0))
    mat_model = glm.rotate(mat_model, angulo, glm.vec3(0.0, 0.0, 1.0))
    mat_model = glm.scale(mat_model, glm.vec3(escala, escala, 1.0))
    
    # Matriz final
    mvp = proj * view * model
    
    # Envia para o shader
    glUniformMatrix4fv(loc_transform, 1, GL_FALSE, glm.value_ptr(mvp))
    
    # Desenha
    glUniform4f(loc_color, 1.0, 1.0, 1.0, 0.75) 
    glDrawElements(GL_TRIANGLE_FAN, len(poligonos_indices), GL_UNSIGNED_INT, None)

    # Descomente as duas linhas abaixo para conseguir ver as arestas
    # glUniform4f(loc_color, 0.0, 0.0, 0.0, 1.0)  # Cor preta
    # glPolygonMode(GL_FRONT_AND_BACK, GL_LINE)    # Modo wireframe
    glDrawElements(GL_TRIANGLES, len(poligonos_indices), GL_UNSIGNED_INT, None)
    glPolygonMode(GL_FRONT_AND_BACK, GL_FILL)    # Volta ao modo sólido
    
    glfw.swap_buffers(window)
    glfw.poll_events()

glfw.terminate()